In [4]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import HumanMessage,BaseMessage,SystemMessage,ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool


load_dotenv()


llm= ChatOpenAI(model="gpt-4o",temperature=0)

embeddings=OpenAIEmbeddings(model="text-embedding-3-small") 

pdf_path="CSE_Today_Analysis.pdf"

if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF file not found at path: {pdf_path}")

pdf_loader=PyPDFLoader(pdf_path)

try:
    pages=pdf_loader.load()
    print(f"pdf has been loaded and has {len(pages)} pages")
except Exception as e:
    print(f"Error loading PDF: {e}")
    raise

#chunking process
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

pages_split=text_splitter.split_documents(pages)

persist_directory=r"D:\LangGraph\projects\basic tasks" # here have to define the path for chromadb
collection_name="stock_market"

if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

try:
    vectorstore=Chroma.from_documents(documents=pages_split,embedding=embeddings,persist_directory=persist_directory,collection_name=collection_name)
    print(f"Created chromadb vectorstore with collection: {collection_name}")
except Exception as e:
    print(f"Error creating vectorstore: {str(e)}")
    raise


retriver=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":5}) # amount of chunks to return. here 5


@tool
def retriver_tool(query:str) -> str:
    """use this tool to retrieve relevant information from the document(CSE today analysis)"""
    docs=retriver.invoke(query)
    
    if not docs:
        return "No relevant information found in the document."
    
    results=[]

    for i,doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}\n")  
    
    return "\n\n".join(results)


tools=[retriver_tool]

llm=llm.bind_tools(tools)

class AgentState(TypedDict):
    messages:Annotated[Sequence[BaseMessage],add_messages]


def should_continue(state:AgentState) -> str:
    """decide whether to continue or end the process based on the last message"""
    result=state["messages"][-1]
    return hasattr(result,"tool_calls") and len(result.tool_calls)>0

system_prompt="""You are intelligent AI assistant who answer questions about Colombo Stock market performance in november 2025 based on the PDF document loaded into your knowledge base. Use the retriver tool available to answer question about the stock market performance data. Your can read the recipe you like. if you need to look up some information befire asking a follow up question, you are allowed to do that. Please always cite the specific parts of the document you use in you answer.
"""

tool_dict={our_tool.name:our_tool for our_tool in tools} #Creating a dictionary of tools for easy access

#LLM Agent
def call_llm(state:AgentState)->AgentState:
    """Function to call the LLM with the current state messages"""
    messages=list(state["messages"])
    messages=[SystemMessage(content=system_prompt)]+messages
    messages=llm.invoke(messages)
    return {"messages":[messages]}

 
#Retriever agent
def take_action(state:AgentState)->AgentState:
    """Execute tool calls from the LLM's response"""
    tool_calls=state["messages"][-1].tool_calls
    results=[]

    for t in tool_calls:
        print(f"calling tool: {t['name']} with query: {t['args'].get('query','No query provided')}")
        if not t['name'] in tool_dict:
            print(f"Tool:({t['name']}) does not exist.")
            result='Incorrect Tool name, Please retry and select tool from the list of available tools'
        else:
            result=tool_dict[t['name']].invoke(t['args'].get('query',''))
            print(f"Result length: {len(str(result))}")
        
        results.append(ToolMessage(tool_call_id=t['id'],name=t['name'],content=str(result)))
    
    print("Tools execution complete. Back to the model!")
    return {'messages':results}


graph=StateGraph(AgentState)      
graph.add_node("llm",call_llm)
graph.add_node("retriver_agent",take_action)

graph.add_conditional_edges(
    "llm",
    should_continue,
    {True:'retriver_agent',False:END}
    )

graph.add_edge("retriver_agent","llm")
graph.set_entry_point("llm")


rag_agent=graph.compile()

def running_agent():
    print("\n=== Rag Agent ===")

    while True:
        user_input=input('\nwhat is your question: ')
        if user_input.lower() in ['exit','quit']:
            break
        
        messages=[HumanMessage(content=user_input)]

        # result=rag_agent({"messages":messages})
        result=rag_agent.invoke({"messages": messages})
        print("\n=== Agent Response ===")
        print(result["messages"][-1].content)


running_agent()



        
        
    





     





pdf has been loaded and has 4 pages
Created chromadb vectorstore with collection: stock_market

=== Rag Agent ===
calling tool: retriver_tool with query: best company to buy in Colombo Stock market November 2025
Result length: 4507
Tools execution complete. Back to the model!

=== Agent Response ===
Based on the data from November 2025, the best companies to consider for buying in the Colombo Stock Market, based on their performance, are:

1. **Kotmale Holdings (LAMB.N0000)**: It had a significant increase in value with a 24.99% change, reaching a VWAP (Volume Weighted Average Price) of 1,608.25 LKR.

2. **Senkadagala (SFCL.N0000)**: This company also showed a strong performance with a 24.92% increase, reaching a VWAP of 1,656.75 LKR.

3. **Bogala Graphite (BOGA.N0000)**: It experienced a 24.83% increase, with a VWAP of 137.00 LKR.

These companies were among the top gainers in the market, indicating strong performance and potential investment opportunities. Always consider conducting 